In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from collections import OrderedDict

c:\Users\User1\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load model (using BitsAndBytes for efficiency)
model_name = "meta-llama/Llama-3.1-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16,
    token=True
)

c:\Users\User1\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\auto\tokenization_auto.py:934: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:50<00:00, 12.52s/it]


In [4]:
#Model Details
print("Model type:", type(model).__name__)
print("Model config:", model.config.model_type)
print("Number of layers:", model.config.num_hidden_layers)
print("Hidden size:", model.config.hidden_size)
print("Number of attention heads:", model.config.num_attention_heads)
print()

# Top-level components
print("Top-level model components:")
for name, module in model.named_children():
    print(f"  {name}: {type(module).__name__}")
print()

# Model layers structure (Llama specific)
if hasattr(model, 'model'):
    print("Core model components:")
    for name, module in model.model.named_children():
        print(f"  model.{name}: {type(module).__name__}")
        if name == "layers":
            print(f"    Number of transformer layers: {len(module)}")
print()

Model type: LlamaForCausalLM
Model config: llama
Number of layers: 32
Hidden size: 4096
Number of attention heads: 32

Top-level model components:
  model: LlamaModel
  lm_head: Linear

Core model components:
  model.embed_tokens: Embedding
  model.layers: ModuleList
    Number of transformer layers: 32
  model.norm: LlamaRMSNorm
  model.rotary_emb: LlamaRotaryEmbedding

